## Model Data using Cassandra

### Introduction

There is a music streaming app called SoundCloud, that has been using their music streaming app and collecting data on songs and user activity and their aim is to analyze this data especially understanding what songs users are listening to. Currently, they are not making use of a NoSQL db and they have the data stored as a CSV file, thus its difficult for them to query the data. So our task is to create a NoSQL database for helping them with the analysis.

#### Importing Packages ::::

In [27]:
import pandas as pd
import numpy as np
import cassandra
from cassandra.cluster import Cluster
import csv

file_name = './data/event_data.csv'

#### Creating a Cluster on local machine and using port 6000 for it::::

In [28]:
# Task: Make a connection to the cassandra instance on your local machine(127.0.0.1) and 
# create a session to establish connection and begin executing queries
try: 
    cluster = Cluster(['127.0.0.1'],  port=6000)
    session = cluster.connect()
    session.execute("""select * from sound_cloud""")
except Exception as err:
    print(err)

Error from server: code=2200 [Invalid query] message="No keyspace has been specified. USE a keyspace, or explicitly specify keyspace.tablename"


#### Creating & Setting Keyspace::::

In [29]:
# Task: Create a Keyspace and Set KEYSPACE to the keyspace specified above
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS nosqldb WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 3 }""")
    session.set_keyspace('nosqldb')
    session.execute("""select * from sound_cloud""")
except Exception as err:
    print(err)

Error from server: code=2200 [Invalid query] message="table sound_cloud does not exist"


## List of Queries 

### 1. Find the artist_name, song_title and length of song the SoundCloud app history that was heard during  session_number = 338, and item_in_session_number  = 4
`Select artist_name, song_title, length from sound_cloud WHERE session_number = 338 and item_in_session_number = 4;`


### Ques: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key for the query 1
#### Table -> sound_could_history_0
* Used session_number as partition key  and item_in_session_number as clustering key

In [30]:
query = """CREATE TABLE IF NOT EXISTS sound_cloud_history_0 (
    user_id int,
    artist_name text,
    song_title text,
    length float,
    session_number int,
    item_in_session_number int,
    PRIMARY KEY (session_number, item_in_session_number))"""
try:
    session.execute(query)
    print("---------Table created------------")
except Exception as e:
    print(e)       

---------Table created------------


### Inserting our data into of table::::::

In [31]:
with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader)
    for row in csv_reader:
        query = """INSERT INTO sound_cloud_history_0 
        (user_id, artist_name, song_title, length, session_number, item_in_session_number)  
        VALUES (%s, %s, %s, %s, %s, %s)"""
        session.execute(query, (int(row[10]), row[0], row[9], float(row[5]), int(row[8]), int(row[3])))

### Validating our Data Model using a SELECT:::::

In [32]:
query = """SELECT artist_name, song_title, length, session_number, item_in_session_number
FROM sound_cloud_history_0 
WHERE session_number = 338 and item_in_session_number = 4;"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

print ("{:<15} {:<25} {:<15} {:<15}".format("session_number", "item_in_session_number", "artist_name", "song_title", "length"))
print ("--------------------------------------------------------------------------------------------------------------------")
for row in rows:
    print ("{:<15} {:<25} {:<15} {:<15}".format(row.session_number, row.item_in_session_number, row.artist_name, row.song_title, row.length))

session_number  item_in_session_number    artist_name     song_title     
--------------------------------------------------------------------------------------------------------------------
338             4                         Faithless       Music Matters (Mark Knight Dub)


### 2. Find the artist_name, song_title (sorted by item_in_session_number) and name(fname and lname) of the user for user_id = 10, session_number = 182
    
`Select artist_name, song_title, fname, lname from sound_cloud Where user_id = 10 and session_number = 182 ORDER BY item_in_session_number;`

### Ques: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key for the query 2
#### Table -> sound_could_history_1
- Used user_id, session_number as partition key and item_in_session_number as clustering key and combination forms a primary key

In [33]:
query = """CREATE TABLE IF NOT EXISTS sound_cloud_history_1 (
    user_id int,
    artist_name text,
    fname text,
    lname text,
    song_title text,
    session_number int,
    item_in_session_number int,
    PRIMARY KEY ((user_id, session_number), item_in_session_number))"""
try:
    session.execute(query)
    print("---------Table created------------")
except Exception as e:
    print(e)                  

---------Table created------------


### Inserting our data into of table::::

In [34]:
with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader)
    for row in csv_reader:
        query = """INSERT INTO sound_cloud_history_1 
        (user_id, artist_name, fname, lname, song_title, session_number, item_in_session_number)  
        VALUES (%s, %s, %s, %s, %s, %s, %s)"""
        session.execute(query, (int(row[10]), row[0], row[1], row[4], row[9], int(row[8]), int(row[3])))


### Validating our Data Model using a SELECT::::

In [35]:
query = """SELECT artist_name, song_title, fname, lname, session_number, item_in_session_number, user_id 
FROM sound_cloud_history_1 
WHERE user_id = 10 and session_number = 182 
ORDER BY item_in_session_number;"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
print ("{:<25} {:<15} {:<15} {:<15} {:<15} {:<25} {:<15}".format("item_in_session_number", "session_number", "user_id", "fname", "lname", "artist_name", "song_title"))
print ("--------------------------------------------------------------------------------------------------------------------------------")

for row in rows:
    print ("{:<25} {:<15} {:<15} {:<15} {:<15} {:<25} {:<15}".format(row.item_in_session_number, row.session_number, row.user_id, row.fname, row.lname, row.artist_name, row.song_title))

item_in_session_number    session_number  user_id         fname           lname           artist_name               song_title     
--------------------------------------------------------------------------------------------------------------------------------
0                         182             10              Sylvie          Cruz            Down To The Bone          Keep On Keepin' On
1                         182             10              Sylvie          Cruz            Three Drives              Greece 2000    
2                         182             10              Sylvie          Cruz            Sebastien Tellier         Kilometer      
3                         182             10              Sylvie          Cruz            Lonnie Gordon             Catch You Baby (Steve Pitron & Max Sanna Radio Edit)


### 3. Find every name(fname and lname) of the user from the SoundCloud app history that listened to the song_title 'All Hands Against His Own'
`SELECT fname, lname FROM sound_cloud WHERE song_title = 'All Hands Against His Own';`

### Ques: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key for the query 3

#### Table -> sound_could_history_2
- Used song_title as partition key and user_id as clustering key and combination forms a primary key

In [36]:
query = """CREATE TABLE IF NOT EXISTS sound_cloud_history_2 (
    user_id int,
    fname text,
    lname text,
    song_title text,
    PRIMARY KEY (song_title, user_id))"""
try:
    session.execute(query)
    print("---------Table created------------")
except Exception as e:
    print(e)               

---------Table created------------


### Inserting our data into of table::::

In [37]:
with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader)
    for row in csv_reader:
        query = """INSERT INTO sound_cloud_history_2 
        (user_id, fname, lname, song_title)  
        VALUES (%s, %s, %s, %s)"""
        session.execute(query, (int(row[10]), row[1], row[4], row[9]))

### Validating our Data Model using a SELECT::::

In [38]:
query = """SELECT user_id,fname, lname, song_title
FROM sound_cloud_history_2 
WHERE song_title = 'All Hands Against His Own';"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
print ("{:<15} {:<15} {:<15} {:<15}".format("user_id", "fname", "lname", "song_title"))
print ("--------------------------------------------------------------------------------------------------------------------------------")
for row in rows:
    print ("{:<15} {:<15} {:<15} {:<15}".format(row.user_id, row.fname, row.lname, row.song_title))

user_id         fname           lname           song_title     
--------------------------------------------------------------------------------------------------------------------------------
29              Jacqueline      Lynch           All Hands Against His Own
80              Tegan           Levine          All Hands Against His Own
95              Sara            Johnson         All Hands Against His Own


### Dropping the tables before closing out the sessions::::

In [39]:
tables = ['sound_cloud_history_0','sound_cloud_history_1','sound_cloud_history_2']
for tb in tables:
  try:
        query = "DROP table" + " " + tb
        session.execute(query)
        print(f"Deleted table {tb}")
  except Exception as e:
      print(e)

Deleted table sound_cloud_history_0
Deleted table sound_cloud_history_1
Deleted table sound_cloud_history_2


### Closing the session and cluster connection::::¶

In [40]:
session.shutdown()
cluster.shutdown()